# Dog Breed Identification by Image Classification


The goal of this project is to be able to identify the breed of a dog given a picture of it. The data was provided by Kaggle for their dog breed identification contest. My approach is to use a convolutional neural network to classify these images. I will train a variety of networks of my own design and additionally apply some pre-trained networks and compare the results. 

In [1]:
#import relevant packages
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, Input, Activation
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.applications import ResNet50
from keras import Model
from keras.preprocessing import image
import cv2
from sklearn.model_selection import train_test_split
from tqdm import tqdm

Using TensorFlow backend.
/home/connor/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
t_labels = pd.read_csv('../data/dogs/data/sample_submission.csv')


In [3]:
t_labels.head(1)

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333


In [4]:
#The images are all different dimensions so this will be the parameter to
#resize them to be square 
img_size = 112

#load the data
labels = pd.read_csv('../data/dogs/data/labels.csv.zip',compression='zip')
t_labels = pd.read_csv('../data/dogs/data/sample_submission.csv')
labels.head(2)

targets_series = pd.Series(labels['breed'])

one_hot = pd.get_dummies(targets_series,sparse = True)

oh_labels = np.asarray(one_hot)
x_train = []
y_train = []
x_test = []

i = 0
for path, breed in tqdm(labels.values):
    img = cv2.imread('../data/dogs/data/train/{}.jpg'.format(path))
    label = oh_labels[i]
    x_train.append(cv2.resize(img,(img_size,img_size)))
    y_train.append(label)
    i+=1
for path in tqdm(t_labels['id'].values):
    img = cv2.imread('../data/dogs/data/test/{}.jpg'.format(path))
    x_test.append(cv2.resize(img,(img_size,img_size)))


100%|██████████| 10357/10357 [00:30<00:00, 337.28it/s]


In [5]:
y_train_raw = np.array(y_train, np.uint8)
x_train_raw = np.array(x_train, np.float32) / 255.
x_test  = np.array(x_test, np.float32) / 255.


In [6]:
number_classes = y_train_raw.shape[1]

In [7]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=0.2, random_state=1)

In [8]:
base_model = VGG16(weights = 'imagenet', include_top = False,
                      input_shape = (img_size,img_size,3))
x = base_model.output
x = Flatten()(x)
predictions = Dense(number_classes, activation = 'softmax')(x)
model = Model(inputs= base_model.input,outputs = predictions)
for layer in base_model.layers:
    layer.trainable = False
model.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics = ['accuracy'])
callbacks_list = [keras.callbacks.EarlyStopping(monitor = 'val_acc',patience = 3,verbose = 1)]

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 112, 112, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 112, 112, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 112, 112, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 56, 56, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 56, 56, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 56, 56, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 28, 28, 128)       0         
__________

In [10]:
model.fit(X_train,Y_train,epochs = 20, validation_data= (X_valid,Y_valid),verbose = 1)

Train on 8177 samples, validate on 2045 samples
Epoch 1/20
7808/8177 [===========================>..] - ETA: 1:07 - loss: 4.3125 - acc: 0.1013

KeyboardInterrupt: 

In [ ]:
preds = model.predict(x_test,verbose = 1)


In [ ]:
t_labels.to

In [ ]:
sub = pd.DataFrame(preds)
col_names = one_hot.columns.values
sub.columns = col_names
sub.insert(0,'id',t_labels['id'])
sub.head()

### Preparing the data to train the networks

In [4]:
def prepare_train_validation():
    #converting breed categories to categorical booleans
    one_hot_labels = pd.get_dummies(labels['breed']).values
    X_raw = []
    y_raw = []
    #loads and processes the images into image_size by image_size by 3 tensors
    for ind, row in enumerate(labels.values):
        img = cv2.imread('../data/dogs/data/train/{}.jpg'.format(row[0]))
        img = cv2.resize(img,(image_size,image_size))
        X_raw.append(img)
        y_raw.append(one_hot_labels[ind,:])
    X = np.array(X_raw)
    y =np.array(y_raw)
    X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.2,stratify = y, random_state=1)
    mean_pixel = np.array([103.939, 116.779, 123.68])
    #training_mean = np.mean(X_train,axis=0)
    X_train = X_train - mean_pixel
    X_validation = X_validation - mean_pixel
    return X_train, X_validation, y_train, y_validation

In [5]:
#creates training and validation sets with every class represented
#in both the training and validation sets.
X_train, X_validation, y_train, y_validation = prepare_train_validation()


MemoryError: 

In [6]:
vgg_16_conv = VGG16(weights='imagenet',include_top = False)
img_in = Input(shape = (image_size,image_size,3),name = 'image_input')
outputVGG16 = vgg_16_conv(img_in)

X = Flatten()(outputVGG16)
X = Dense(4096, activation='relu',name="dense1")(X)
X = Dense(4096, activation='relu',name="dense2")(X)
X = Dense(120, activation='softmax', name="output")(X)
adapted_VGG16 = Model(input = img_in,output = X)

Ad = Adam(lr = .01 , beta_1 = .9, beta_2 = .999, epsilon = 1e-08, decay =0.0)
adapted_VGG16.compile(optimizer =Ad, loss = 'categorical_crossentropy',metrics = ['accuracy'] )

history = adapted_VGG16.fit(X_train,y_train,epochs = 1, batch_size = 19)
results = adapted_VGG16.evaluate(X_validation,y_validation)

/home/connor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("im..., outputs=Tensor("ou...)`
  if __name__ == '__main__':


NameError: name 'X_train' is not defined

## First Experimental Network
this network is based on VGG16 but has been simplified 

In [ ]:
def conv_network_1( input_shape,weights_path = None):
    model = Sequential()
    
    model.add(ZeroPadding2D((1,1),input_shape=input_shape))
    
    model.add(Convolution2D(32,(3,3),activation ='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(32,(3,3),activation ='relu'))
    #model.add(MaxPooling2D((2,2),strides = (2,2)))
    
    model.add(Convolution2D(64,(3,3),activation ='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64,(3,3),activation ='relu'))
    model.add(MaxPooling2D((2,2),strides = (2,2)))
    
    model.add(Convolution2D(128,(3,3),activation ='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128,(3,3),activation ='relu'))
    model.add(MaxPooling2D((2,2),strides = (2,2)))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu',name="dense1"))
    model.add(Dense(1024, activation='relu',name="dense2"))
    model.add(Dense(120, activation= 'softmax',name="dense3"))
    return model

In [ ]:
#Training the network using an rmsprop optimizer
model = conv_network_1(input_shape = (image_size,image_size,3))
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy',metrics = ['accuracy'] )
model.fit(X_train,y_train,epochs = 10, batch_size = 19)

In [ ]:
#for layer in model.layers:
#    print(layer.get_output_at(0).get_shape().as_list())
#model.fit(X,y,epochs = 10, batch_size = 38)

In [ ]:
def conv_network_2(input_shape):
    X_input = Input(input_shape)
    X = ZeroPadding2D((1,1),input_shape=input_shape)(X_input)
    
    X = Convolution2D(32,(5,5),activation ='relu',name='conv1a')(X)
    X = ZeroPadding2D((1,1))(X)
    X = Convolution2D(32,(5,5),activation ='relu',name='conv1b')(X)
    X = MaxPooling2D((2,2),strides = (2,2),name = 'pool1a')(X)
    
    X = Convolution2D(64,(3,3),activation ='relu',name='conv2a')(X)
    X = ZeroPadding2D((1,1))(X)
    X = Convolution2D(64,(3,3),activation ='relu',name='conv2b')(X)
    X = MaxPooling2D((2,2),strides = (2,2),name = 'pool2a')(X)
    
    X = Convolution2D(128,(3,3),activation ='relu',name='conv3a')(X)
    X = ZeroPadding2D((1,1))(X)
    X = Convolution2D(128,(3,3),activation ='relu',name='conv3b')(X)
    X = MaxPooling2D((2,2),strides = (2,2),name = 'pool3a')(X)
    X = Activation('relu')(X)
    
    X = Flatten()(X)
    X = Dense(120, activation='relu',name="dense1")(X)

    model = Model(inputs = X_input, outputs = X, name = 'convnet2')
    return model

In [ ]:
#model2 = conv_network_2(X.shape[1:])

In [ ]:
#model2.compile(optimizer = 'rmsprop',loss = 'categorical_crossentropy',)

In [ ]:
#model2.fit(X,y,epochs = 10, batch_size = 38)